#SpaceX - Falcon 9: Interactive Visual Analytics with Folium

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

- In the previous exploratory data analysis notebook, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates.

- In this lab, you will be performing more interactive visual analytics using `Folium`.

## Import Libraries

In [1]:
import pandas as pd
import folium
# Import folium MarkeCuster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

# Mark all launch sites on a map

The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.

In [2]:
# Download and read the 'spacex_Lauch_geo.csv'
spacex_csv_file = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"
spacex_df = pd.read_csv(spacex_csv_file)

spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [3]:
# Select relevant sub-columns: "Launch Site", "Lat(Latitude)", "Long(Longitude)", "class"
spacex_df = spacex_df[["Launch Site", "Lat", "Long", "class"]]
launch_sites_df = spacex_df.groupby(["Launch Site"], as_index=False).first()
launch_sites_df = launch_sites_df[["Launch Site", "Lat", "Long"]]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [4]:
# Start Location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,

In [6]:
# Create a blue circle as NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup("NASA Johnson Space Center"))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size:12; color:#d35400;"><b>%s</b></div>' % "NASA JSC",
    )
)
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small orange circle near the city of Houston and you can zoom-in to see a larger circle.

Now, let's add a circle for each launch site in data frame `launch_sites`

In [7]:
# Initial th map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for lat, lng, label in zip(launch_sites_df["Lat"], launch_sites_df["Long"], launch_sites_df["Launch Site"]):
    coordinate = [lat, lng]
    circle = folium.Circle(coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(label))
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size:12; color:#d35400;"><b>%s</b></div>' % label,
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

Explore the map by zoom-in/out the marked areas
we answer the following questions:

*   Are all launch sites in proximity to the Equator line?
 - Response: **CCAFS LC - 40**
*   Are all launch sites in very close proximity to the coast?
 - Response: **CCAFS LC - 40** and **VAFB SLC 4E**

# Step 2: Mark the success/failed launches for each site on the map

Enhancing the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not

In [8]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

Let's first create a `MarkerCluster` object

In [9]:
marker_cluster = MarkerCluster()

In [10]:
'''Applying a function to check the value of `class` column
  If class=1, marker_color value will be green
  If class=0, marker_color value will be red
'''
# Adding narker_cluster to curent site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    coordinate = [record["Lat"], record["Long"]]
    if record["class"] == 1:
        marker_color = "green"
    else:
        marker_color = "red"
    # marker = folium.Marker(...)
    marker = folium.Marker(
        coordinate,
        icon=folium.Icon(color="white", icon_color=marker_color)
    )
    marker_cluster.add_child(marker)

site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.

## Step 3: Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [19]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

In [20]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

-  Finding coordinate of the closet coastline
-  e.g.,: Lat: 28.56367  Lon: -80.57163
-  distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [21]:
# Define a function to find the coordinate of the closest coastline
def find_closest_coastline(launch_site_lat, launch_site_lon):
    """
    Finds the coordinate of the closest coastline to a given launch site.

    Args:
        launch_site_lat: Latitude of the launch site.
        launch_site_lon: Longitude of the launch site.

    Returns:
        A tuple containing the latitude and longitude of the closest coastline,
        or None if no coastline data is available.
        Also returns the distance to the coastline.
    """
    # Placeholder for coastline data (replace with actual coastline data)
    # This example uses hardcoded coordinates for demonstration.
    coastline_data = {
        "coordinates": [(28.56367, -80.57163)]  # Example coastline point
        # Add more coastline coordinates here as needed.
    }

    if not coastline_data or not coastline_data["coordinates"]:
      return None, None

    closest_coastline = None
    min_distance = float('inf')
    for coastline_lat, coastline_lon in coastline_data["coordinates"]:
        distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
        if distance < min_distance:
            min_distance = distance
            closest_coastline = (coastline_lat, coastline_lon)

    return closest_coastline, min_distance

In [22]:
# Create and add a folium.Marker on your selected closest coastline point on the map
'''Display the distance between coastline point and launch site using the icon property '''

# Find the closest coastline for each launch site and add a marker
for index, record in launch_sites_df.iterrows():
    launch_site_lat = record["Lat"]
    launch_site_lon = record["Long"]
    closest_coastline, distance = find_closest_coastline(launch_site_lat, launch_site_lon)

    if closest_coastline:
        coordinate = closest_coastline
        distance_marker = folium.Marker(
            coordinate,
            icon=DivIcon(
                icon_size=(20,20),
                icon_anchor=(0,0),
                html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
            )
        )
        site_map.add_child(distance_marker)
site_map

Draw a `PolyLine` between a launch site to the selected coastline point

In [23]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
coordinates = [(28.561857, -80.577366), (28.56367, -80.57163)]

lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


A railway map symbol may look like this:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


A highway map symbol may look like this:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>

A city map symbol may look like this:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>

- Create a marker with distance to a closest city, railway, highway, etc.
- Draw a line between the marker to the launch site

In [24]:
# # Draw a line between the marker to the launch site
# Function to find the coordinate of the closest landmark (city, railway, highway)
def find_closest_landmark(launch_site_lat, launch_site_lon, landmark_type):
    """
    Finds the coordinate of the closest landmark to a given launch site.

    Args:
        launch_site_lat: Latitude of the launch site.
        launch_site_lon: Longitude of the launch site.
        landmark_type: Type of landmark ("city", "railway", "highway").

    Returns:
        A tuple containing the latitude, longitude, and distance to the closest landmark.
        Returns None if no landmark data is available.
    """

    # Placeholder landmark data (replace with your actual landmark data)
    landmark_data = {
        "city": {
            "coordinates": [(28.5383, -80.6004)]  # Example city coordinates
        },
        "railway": {
            "coordinates": [(28.5718, -80.5852)]  # Example railway coordinates
        },
        "highway": {
            "coordinates": [(28.5734, -80.5678)]  # Example highway coordinates
        }
    }

    if landmark_type not in landmark_data or not landmark_data[landmark_type]["coordinates"]:
      return None, None, None

    closest_landmark = None
    min_distance = float('inf')
    for landmark_lat, landmark_lon in landmark_data[landmark_type]["coordinates"]:
        distance = calculate_distance(launch_site_lat, launch_site_lon, landmark_lat, landmark_lon)
        if distance < min_distance:
            min_distance = distance
            closest_landmark = (landmark_lat, landmark_lon)

    return closest_landmark, min_distance, landmark_type

# Add markers and lines for closest landmarks
for index, record in launch_sites_df.iterrows():
    launch_site_lat = record["Lat"]
    launch_site_lon = record["Long"]

    for landmark_type in ["city", "railway", "highway"]:
        closest_landmark, distance, landmark_type = find_closest_landmark(launch_site_lat, launch_site_lon, landmark_type)
        if closest_landmark:
            coordinate = closest_landmark

            # Customize marker icon based on the landmark type
            if landmark_type == "city":
                icon_html = '<div style="font-size: 12; color:blue;"><b>City</b></div>'
            elif landmark_type == "railway":
                icon_html = '<div style="font-size: 12; color:black;"><b>Railway</b></div>'
            elif landmark_type == "highway":
                icon_html = '<div style="font-size: 12; color:gray;"><b>Highway</b></div>'

            distance_marker = folium.Marker(
                coordinate,
                icon=DivIcon(
                    icon_size=(20, 20),
                    icon_anchor=(0, 0),
                    html=icon_html,  # Add the icon_html here
                )
            )
            site_map.add_child(distance_marker)

            # Draw a line to the landmark
            coordinates = [(launch_site_lat, launch_site_lon), closest_landmark]
            lines = folium.PolyLine(locations=coordinates, weight=1, color="gray") # Custom color
            site_map.add_child(lines)

site_map

After you plot distance lines to the proximities, you can answer the following questions easily:
*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

- Also please try to explain your findings.


# Analysis of Launch Site Proximities

Based on the provided map and the calculated distances to the nearest coastline, city, railway, and highway, we can analyze the proximity of launch sites to these features.

1. **Proximity to Railways and Highways:**
The map shows the distances to the nearest railway and highway.  We would need to analyze the specific distances for each launch site to definitively answer whether they are in "close proximity."  A threshold distance would need to be defined to classify "close" proximity.  Without such a threshold, we can only visually inspect the map.  In general, launch sites appear relatively close to both railways and highways.

2. **Proximity to Coastline:**
Similarly, the distances to the nearest coastline are displayed for each site.  The analysis would again depend on a defined threshold for what constitutes "close proximity". Visually, some launch sites appear to be close to the coast, while others are further inland.

3. **Distance from Cities:**
The map provides distances to nearby cities.  As with the previous points, a threshold is crucial for determining whether the distance is significant.  We need to know if there is a minimum distance requirement from cities for safety or regulatory reasons. Visually, most launch sites appear to be a moderate distance from cities.

4. **Overall Findings and Potential Considerations:**
 * **Accessibility:** The proximity of highways and railways suggests that launch sites are strategically located for easy access for personnel, equipment, and materials.
 * **Safety Buffer:** The distances from cities might suggest a safety buffer zone around the launch site, ensuring that potential hazards from launches have minimal impact on populated areas.
 * **Environmental Impact:** Proximity to the coastline might raise concerns about the environmental impact of launch operations. A further analysis would be needed to investigate potential impacts on marine life and coastal ecosystems.
 * **Regulatory Compliance:**  The distances to different features likely reflect regulatory requirements and safety standards for launch operations.  Comparing these distances across different launch sites could reveal varying regulatory environments.
 * **Further Analysis Needed:** The most important limitation is the lack of defined thresholds for what constitutes "close proximity." The analysis is currently qualitative, relying primarily on visual inspection of the map.  To provide a quantitative analysis,  clear criteria for distance thresholds should be established.  Additionally, having access to more complete data (e.g. city boundaries, actual railway lines, detailed coastline maps) rather than point estimates, will lead to more accurate measurements and a better understanding of proximity.

In conclusion, while the map provides a visualization of the proximities, a quantitative analysis using clearly defined distance thresholds and more detailed geospatial data is needed for definitive conclusions.